In [4]:
import numpy as np
import pandas as pd
import math as mt
from PIL import Image
import urllib
from io import StringIO
import matplotlib.pyplot as plt


im = Image.open("mona_lisa.gif")

plt.imshow(im, cmap='gray')
plt.show()




FileNotFoundError: [Errno 2] No such file or directory: 'mona_lisa.gif'

In [3]:
def process_SSVD(file_name, grayscale=True):
    if file_name:
        img = Image.open(file_name)
    else:
        raise Exception("no source")
    arr = np.array(img)
    new = Image.fromarray(arr)
    new.show()
    
process_SSVD("mona_lisa.gif")

FileNotFoundError: [Errno 2] No such file or directory: 'mona_lisa.gif'

In [0]:
# A = np.array([
#     [0, 200, 200, 200, 0],
#     [50, 255, 50, 255, 50],
#     [200, 50, 200, 50, 200],
#     [50, 255, 255, 255, 50],
#     [0, 100, 100, 100, 0],]
# )
A = np.random.randint(256, size=(100, 100))
#A = np.array([[1, 2, 3], [4, 5, 6]])

In [88]:
def shuffle_arr(A, block_size=(16, 16)):
    M, N = A.shape
    m, n = block_size
    height, width = ((M - (M % m)) * (N - (N % n))) // (m * n), m * n
    X = np.zeros((height, width))
    for i in range(height):
        s_x, s_y = (i * n) % (N - (N % n)), i // m * m
        f_x, f_y = s_x + n, s_y + m
        TMP = A[s_y:f_y, s_x:f_x]
        if TMP.shape != block_size:
            continue
        X[i] = TMP.reshape(1, -1)
    return X


print(A)
A = shuffle_arr(A, block_size=(16, 16))
print(A)

[[181  42  57 ..., 217 180 195]
 [243 192  99 ..., 154 141 190]
 [ 10 143  31 ...,  96 224 231]
 ..., 
 [ 65 149 150 ..., 168  42  38]
 [199 204 231 ...,  44 146 134]
 [173 119 245 ..., 100 144  10]]
[[ 181.   42.   57. ...,  194.   63.  224.]
 [ 156.    8.  219. ...,   49.  104.  163.]
 [  26.  220.  101. ...,   81.   78.  107.]
 ..., 
 [  45.  250.   60. ...,  218.   95.  246.]
 [  58.  217.  239. ...,  234.  207.  225.]
 [  75.  166.  143. ...,   11.   24.  219.]]


In [89]:
print(A.shape)

(36, 256)


In [90]:
AAT = np.dot(A, A.T)
ATA = np.dot(A.T, A)

eigvals_ATA, eigvecs_ATA = np.linalg.eig(ATA)

eigvecs_ATA = eigvecs_ATA.T 
eigvecs_ATA = np.array([x for _, x in sorted(zip(eigvals_ATA, eigvecs_ATA), key=lambda pair: pair[0], reverse=True)])
eigvecs_ATA = eigvecs_ATA.T

eigvals_ATA = np.array([x for x in eigvals_ATA if x > 1e-8])
sing_ATA = np.sqrt(eigvals_ATA)

def dot_D_A(d, c):
  return np.array([c[i] * d[i] for i in range(len(d))])
  
  
  A = A[:d]
  for i in range(len(A)):
    A[i] *= D[i]
  return A

S = np.array(sorted(sing_ATA, reverse=True))  # HURRAY

print(S)
print(eigvecs_ATA)

VT = eigvecs_ATA.T  # HURRAY
# np.linalg.inv(dot_D_A(S, VT))

[ 12387.64397693+0.j   2347.14985834+0.j   2202.86907324+0.j
   2127.73306231+0.j   2082.73020600+0.j   2003.84739848+0.j
   1920.72228698+0.j   1838.40100130+0.j   1757.47275157+0.j
   1699.00017925+0.j   1622.90179521+0.j   1541.40199908+0.j
   1249.28742733+0.j   1203.53760983+0.j   1125.23128913+0.j
   1093.61350696+0.j]
[[ 0.06104797+0.j          0.12467432+0.j          0.00408722+0.j         ...,
  -0.02076547+0.j         -0.01411098-0.01668302j -0.01411098+0.01668302j]
 [ 0.07393428+0.j         -0.05509968+0.j          0.02775158+0.j         ...,
   0.00878972+0.j          0.02432382-0.00632971j  0.02432382+0.00632971j]
 [ 0.05604287+0.j         -0.05350544+0.j          0.01100981+0.j         ...,
   0.07444304+0.j         -0.01957105-0.05428245j -0.01957105+0.05428245j]
 ..., 
 [ 0.07138670+0.j          0.04699453+0.j         -0.06755334+0.j         ...,
  -0.06003092+0.j          0.06313766-0.0179617j   0.06313766+0.0179617j ]
 [ 0.05055578+0.j          0.03214992+0.j         

In [91]:
UT = np.zeros((len(S), len(A)))
for i in range(len(S)):
    d = np.dot((1 / S[i]), A)
    UT[i] = np.dot(d, VT[i])
U = UT.T  # HURRAY

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:4: ComplexWarning: Casting complex values to real discards the imaginary part
  after removing the cwd from sys.path.


In [92]:
print("EIG", eigvals_ATA)
print("-" * 10)
print("U", U)
print("SING", S)
print("VT", VT)

('EIG', array([  1.53453723e+08+0.j,   5.50911246e+06+0.j,   4.85263215e+06+0.j,
         4.52724798e+06+0.j,   4.33776511e+06+0.j,   4.01540440e+06+0.j,
         3.68917410e+06+0.j,   3.37971824e+06+0.j,   3.08871047e+06+0.j,
         2.88660161e+06+0.j,   2.63381024e+06+0.j,   2.37592012e+06+0.j,
         1.56071908e+06+0.j,   1.44850278e+06+0.j,   1.26614545e+06+0.j,
         1.19599050e+06+0.j]))
----------
('U', array([[  1.65841542e-01,   3.66863947e-01,  -2.08242444e-01,
          1.38510931e-02,  -1.07366221e-01,   1.50853672e-02,
          1.02043175e-01,   2.60343517e-01,  -1.11314233e-01,
         -6.96645678e-02,  -6.52844265e-02,   1.01064746e-01,
          2.19166758e-02,   5.52519518e-02,   3.76319585e-03,
         -4.96984061e-02],
       [  1.64358696e-01,  -1.74959942e-01,  -2.20603283e-02,
          8.66464942e-02,   8.70782234e-02,  -4.47210376e-01,
          4.81551564e-02,   4.02850076e-02,  -7.82548002e-03,
         -8.41735040e-02,  -8.76378484e-02,   1.89706689

In [93]:
A_RECOVERED = np.dot(U, dot_D_A(S, VT))
print(A_RECOVERED)

[[ 181.+0.j   42.+0.j   57.+0.j ...,  194.+0.j   63.+0.j  224.+0.j]
 [ 156.+0.j    8.+0.j  219.+0.j ...,   49.+0.j  104.+0.j  163.+0.j]
 [  26.+0.j  220.+0.j  101.+0.j ...,   81.+0.j   78.+0.j  107.+0.j]
 ..., 
 [  45.+0.j  250.+0.j   60.+0.j ...,  218.+0.j   95.+0.j  246.+0.j]
 [  58.+0.j  217.+0.j  239.+0.j ...,  234.+0.j  207.+0.j  225.+0.j]
 [  75.+0.j  166.+0.j  143.+0.j ...,   11.+0.j   24.+0.j  219.+0.j]]


In [94]:
print(A_RECOVERED)

[[ 181.+0.j   42.+0.j   57.+0.j ...,  194.+0.j   63.+0.j  224.+0.j]
 [ 156.+0.j    8.+0.j  219.+0.j ...,   49.+0.j  104.+0.j  163.+0.j]
 [  26.+0.j  220.+0.j  101.+0.j ...,   81.+0.j   78.+0.j  107.+0.j]
 ..., 
 [  45.+0.j  250.+0.j   60.+0.j ...,  218.+0.j   95.+0.j  246.+0.j]
 [  58.+0.j  217.+0.j  239.+0.j ...,  234.+0.j  207.+0.j  225.+0.j]
 [  75.+0.j  166.+0.j  143.+0.j ...,   11.+0.j   24.+0.j  219.+0.j]]


In [0]:
def resuffle_arr(X, A_size, block_size):
  pass